In [1]:
from htmd.ui import *
from moleculekit.config import config
from moleculekit.util import maxDistance
from htmd.home import home
from os.path import join

config(viewer='ngl')

2024-12-05 13:52:37,309 - numexpr.utils - INFO - Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-12-05 13:52:37,310 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
/home/sdoerr/miniforge3/envs/htmd/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049
HTMD Documentation at: https://software.acellera.com/htmd/

You are on the latest HTMD version (2.4.2+23.g4c98f4164.dirty).



# Using docking to generate starting poses for simulations

Download the files for this tutorial from this [link](http://pub.htmd.org/nc983hu3brda/bentryp.tar.gz)

## Dock the protein with the ligand

In [2]:
datadir = join(home(dataDir="building-protein-ligand"))

prot = Molecule(join(datadir, "trypsin.pdb"))
prot.center()
lig = Molecule(join(datadir, "BEN.cif"))
poses, scores = dock(prot, lig)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /tmp/tmpmhkzxhom/protein.pdb)

1 molecule converted
2024-12-05 13:52:42,850 - htmd.dock - INFO - Charges detected in ligand and will be used for docking.
1 molecule converted


#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 20 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: 12

20 molecules converted
20 files output. The first is /tmp/tmpmhkzxhom/output_1.pdb


### Visualize the docked poses

In [3]:
mol = Molecule()
mol.append(prot)
for i, p in enumerate(poses):
    mol.append(p)
mol.view(sel='protein', style='NewCartoon', hold=True)
mol.view(sel='resname BEN', style='Licorice', color=1)

NGLWidget()

## Build systems from docked poses

In [ ]:
molbuilt = []

for i, p in enumerate(poses):
    prot = Molecule(join(datadir, "trypsin.pdb"))
    prot.filter('chain A and (protein or water or resname CA)')
    prot.set('segid', 'P', sel='protein and noh')
    prot.set('segid', 'W', sel='water')
    prot.set('segid', 'CA', sel='resname CA')
    prot.center()
    
    D = maxDistance(prot, 'all')
    
    ligand = p
    ligand.set('segid','L')
    
    mol = Molecule(name='combo')
    mol.append(prot)
    mol.append(ligand)
    
    D = D + 15
    smol = solvate(mol, minmax=[[-D, -D, -D], [D, D, D]])
    topos  = [join(datadir, "BEN.cif")]
    params = [join(datadir, "BEN.frcmod")]

    molbuilt.append(amber.build(smol, topo=topos, param=params, outdir=f'./docked/build/{i+1}/', saltconc=0.15))
    if i==1: # For demonstration purposes lets only build the two first
        break

2024-12-05 13:53:44,922 - moleculekit.molecule - INFO - Removed 9 atoms. 1692 atoms remaining in the molecule.
2024-12-05 13:53:44,952 - htmd.builder.solvate - INFO - Using water pdb file at: /home/sdoerr/Work/htmd/htmd/share/solvate/wat.pdb
2024-12-05 13:53:45,252 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2

2024-12-05 13:53:46,435 - htmd.builder.solvate - INFO - 19297 water molecules were added to the system.
2024-12-05 13:53:48,964 - htmd.builder.amber - INFO - Detecting disulfide bonds.
2024-12-05 13:53:48,981 - htmd.builder.builder - INFO - 6 disulfide bonds were added


Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 26, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 42, insertion: '', segid: 'P'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 117, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 184, insertion: '', segid: 'P'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 174, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 198, insertion: '', segid: 'P'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 149, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 163, insertion: '', segid: 'P'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 8, insertion: '', segid: 'P'>
                   and: UniqueResidue

2024-12-05 13:53:50,234 - htmd.builder.amber - INFO - Starting the build.
2024-12-05 13:53:53,664 - htmd.builder.amber - INFO - Finished building.
2024-12-05 13:53:54,470 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.
2024-12-05 13:53:59,698 - htmd.builder.ionize - INFO - Adding 9 anions + 0 cations for neutralizing and 108 ions for the given salt concentration 0.15 M.
2024-12-05 13:54:11,829 - htmd.builder.amber - INFO - Starting the build.
2024-12-05 13:54:14,825 - htmd.builder.amber - INFO - Finished building.
2024-12-05 13:54:15,564 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.
/home/sdoerr/miniforge3/envs/htmd/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a 

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 26, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 42, insertion: '', segid: 'P'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 117, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 184, insertion: '', segid: 'P'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 174, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 198, insertion: '', segid: 'P'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 149, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 163, insertion: '', segid: 'P'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 8, insertion: '', segid: 'P'>
                   and: UniqueResidue

2024-12-05 13:54:28,367 - htmd.builder.amber - INFO - Starting the build.
2024-12-05 13:54:31,500 - htmd.builder.amber - INFO - Finished building.
2024-12-05 13:54:32,248 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.
2024-12-05 13:54:37,561 - htmd.builder.ionize - INFO - Adding 9 anions + 0 cations for neutralizing and 108 ions for the given salt concentration 0.15 M.


## Equilibrate the build systems

In [ ]:
from acemd.protocols import setup_equilibration

builds = sorted(glob('docked/build/*/'))
for i, bd in enumerate(builds):
    setup_equilibration(bd, f'docked/equil/{i+1}/', run="10ns", temperature=298)

In [ ]:
mdx = LocalGPUQueue()
mdx.submit(glob('./docked/equil/*/'))

In [ ]:
mdx.wait()

## Create the production folder

In [ ]:
from acemd.protocols import setup_production

equils = sorted(glob('docked/equil/*/'))
for i, eq in enumerate(equils):
    setup_production(eq, f'docked/generators/{i+1}/', run="50ns", temperature=298)

In [ ]:
mdx = LocalGPUQueue()
mdx.submit(glob('./docked/generators/*/'))

In [ ]:
mdx.wait()